# Курсовая работа


## Выполнил студент группы БФИ2204 Данилочкин Дмитрий Алексеевич
***

### Задание

Написать программу, которая будет считывать данные из CSV файла, содержащего информацию о продажах товаров в магазине. Данные в файле содержатся в следующем формате: |Номер заказа | Дата заказа | Название товара | Категория товара | Количество продаж | Цена за единицу | Общая стоимость |

Необходимо:
1. Рассчитать общую выручку магазина.
2. Найти товар, который был продан наибольшее количество раз.
3. Найти товар, который принес наибольшую выручку.
4. Составить отчет, содержащий информацию об общей выручке магазина, количестве проданных единиц каждого товара и доле каждого товара в общей выручке.
Для решения задач необходимо использовать структуры данных, такие как массивы и хеш-таблицы, а также различные алгоритмы обработки данных, например, сортировку и поиск. Также необходимо учитывать возможные ошибки ввода-вывода и обрабатывать их в соответствии с требованиями.


### Выполнение:

In [1]:
#%%capture
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import csv
from datetime import datetime
import math
import random
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox


def read_data():
    total_money = 0
    products = {}
    date_start,date_end = "",""

    try:
        with open("data.csv", "r") as file:
            reader = csv.reader(file, delimiter=";")
            next(reader)  #пропуск заголовока
            for row in reader:
                order_number = int(row[0])
                order_date = datetime.strptime(row[1], "%d.%m.%Y")
                prod_name = row[2]
                prod_category = row[3]
                order_prod_quantity = int(row[4])
                single_prod_money = float(row[5])
                order_money = float(row[6])


                if date_start == "":
                    date_start,date_end = order_date,order_date

                if order_date<date_start:
                    date_start=order_date
                elif order_date>date_end:
                    date_end=order_date

                #общая выручка магазина
                total_money += order_money
                
                order_prod_money = single_prod_money*order_prod_quantity
                if is_found(products, prod_name):
                    products[prod_name][0]+=order_prod_quantity
                    products[prod_name][1]+=order_prod_money
                else:
                    products[prod_name]=[order_prod_quantity,order_prod_money,"portion %",prod_category]


            max_quantity = [0,""]
            max_money = [0,""]
            
            for prod_name in products:
                #товар, который был продан наибольшее количество раз
                total_prod_quantity = products[prod_name][0]
                if total_prod_quantity > max_quantity[0]:
                    max_quantity = [total_prod_quantity,prod_name]

                #товар, который принес наибольшую выручку
                total_prod_money = products[prod_name][1]
                if total_prod_money > max_money[0]:
                    max_money = [total_prod_money,prod_name]
            
                #доля каждого товара в общей выручке
                portion = total_prod_money/total_money * 100
                portion=math.floor(portion)
                products[prod_name][2] = portion

            date_diff = date_end-date_start
            date_start=date_start.strftime("%d.%m.%Y")
            date_end=date_end.strftime("%d.%m.%Y")
            date_diff="Дней: "+str(date_diff.days)
    
    except IOError as e:
        messagebox.showerror("Ошибка",'Не удалось прочитать файл "data.csv"')
        return total_money,products,date_start,date_end,"",[0,""],[0,""]
    
    global label
    label.config(text =
                 "Период: с "+date_start+" по "+date_end+" ("+date_diff+")"
                +"\nОбщая выручка магазина: "+"{:.2f}".format(total_money)
                +"\nТовар "+'«'+max_quantity[1]+'»'+" был продан наибольшее количество раз: "+str(max_quantity[0])
                +"\nТовар "+'«'+max_money[1]+'»'+" принес наибольшую выручку: "+"{:.2f}".format(max_money[0]))

    return total_money,products,date_start,date_end,date_diff,max_quantity,max_money


# Быстрая сортировка
def quick_sort(m, column):
    m_len = len(m)
    if m_len < 2:
        return m

    low,same,high=[],[],[]
    pivot = m[random.randint(0,m_len-1)][column]
    
    for elem in m:
        if elem[column]<pivot:
            low.append(elem)
        elif elem[column]==pivot:
            same.append(elem)
        elif elem[column]>pivot:
            high.append(elem)
    
    return quick_sort(low,column) + same + quick_sort(high,column)

def quick_sort_(m, column):
    arr=[]
    for key in m:
        arr.append([key]+m[key])

    arr = quick_sort(arr, column)

    res = {}
    for elem in arr:
        res[elem[0]]=[elem[1],elem[2],elem[3],elem[4]]

    return res

# Расположение элементов массива в обратном порядке
def invert_arr(m):
    arr=[]
    for key in m:
        arr.append([key]+m[key])
    
    len_ = len(arr)
    new_arr = [None]*len_
    for i in range(len_):
        new_arr[i] = arr[len_ - i - 1]

    res = {}
    for elem in new_arr:
        res[elem[0]]=[elem[1],elem[2],elem[3],elem[4]]

    return res

# Проверка на существование ключа в хеш-таблице
def is_found(tab, key):
    for k in tab:
        if k == key:
            return True
    return False


### Составление отчета ###
def build_report(products):
    total_money,products_,date_start,date_end,date_diff,max_quantity,max_money = read_data()
    products = products or products_

    from reportlab.lib import colors
    from reportlab.lib.pagesizes import letter
    from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Spacer
    from reportlab.pdfbase import pdfmetrics
    from reportlab.pdfbase.ttfonts import TTFont

    data=[["Название товара","Количество\nпродаж","Выручка","Доля в общей\nвыручке"]]
    for prod_name in products:
        products[prod_name][1] = "{:.2f}".format(products[prod_name][1])
        portion = products[prod_name][2]
        if portion<1:
            portion = "< 1"
        portion=str(portion)+" %"
        products[prod_name][2] = portion

        products[prod_name].pop() #убрать категории

        data.append([prod_name]+products[prod_name])

    pdfmetrics.registerFont(TTFont('DejaVuSerif', 'DejaVuSerif.ttf'))

    pdf = SimpleDocTemplate("report.pdf", pagesize=letter)

    table = Table(data)

    style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
                        ('ALIGN', (1, 1), (3, -1), 'RIGHT'),
                        ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                        ('GRID', (0, 0), (-1, -1), 1, colors.black),
                        ('FONTNAME', (0, 0), (-1, -1), 'DejaVuSerif'),
                        ('FONTSIZE', (0, 0), (-1, -1), 13)])
    table.setStyle(style)

    table0  = Table([
        ["Отчет о продажах товаров в магазине\n"],
        ["Отчет о продажах с "+date_start+" по "+date_end+" ("+date_diff+")"+
        "\nОбщая выручка магазина: "+"{:.2f}".format(total_money)+
        "\nТовар "+'«'+max_quantity[1]+'»'+" был продан наибольшее количество раз: "+str(max_quantity[0])+
        "\nТовар "+'«'+max_money[1]+'»'+" принес наибольшую выручку: "+"{:.2f}".format(max_money[0])]

    ])
    table0.setStyle([('FONTNAME', (0, 0), (-1, -1), 'DejaVuSerif'),
                    ('FONTSIZE', (0, 0), (-1, -1), 14),
                    ('LEADING', (0, 0), (-1, -1), 25),
                    ('ALIGN', (0, 0), (0, 0), 'CENTER')
                    ])

    elements = [table0, Spacer(1, 15), table]
    pdf.build(elements)

    messagebox.showinfo("Запись в PDF",'Отчет составлен и записан в "report.pdf"')



### Построение диаграмм ###
canvas = None
def build_pie(type_):
    global canvas
    if canvas:
        canvas.get_tk_widget().destroy()
    canvas = None

    products = list(read_data())[1]
    labels = []
    sizes = []
    categories = {}
    
    for prod_name in products:
        prod_count = products[prod_name][0]
        prod_money = products[prod_name][1]
        prod_portion = products[prod_name][2]
        category = products[prod_name][3]
        if is_found(categories,category):
            categories[category][0]+=prod_count
            categories[category][1]+=prod_money
            categories[category][2]+=prod_portion
        else:
            categories[category] = [prod_count,prod_money,prod_portion]
    
    for category in categories:
        labels.append(category)
        sizes.append(categories[category][type_])

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%')
    ax.axis('equal')

    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    canvas.get_tk_widget().pack()


### Интерфейс ###
window = tk.Tk()
window.configure(background='white')
window.title("Анализ данных о продажах товаров в магазине")
w = window.winfo_screenwidth()
h = window.winfo_screenheight()

x = (w / 2) - (800/2)
y = (h / 2) - (600/2)
window.geometry("800x600+{}+{}".format(int(x), int(y)))

window.resizable(False, False)

label = ttk.Label(window, text="Выберите действие")
label.configure(background='white')
label.pack(side=tk.TOP, anchor=tk.NW)

menubar = tk.Menu(window)
window.config(menu=menubar)

## Выход ##
menubar.add_command(label="Выход", command=window.destroy)

## Построить диаграмму ##
graph_menu = tk.Menu(menubar, tearoff=0)
menubar.add_cascade(label="Построить диаграмму", menu=graph_menu)
graph_menu.add_command(label="По выручке", command=lambda:build_pie(1))
graph_menu.add_command(label="По кол-ву проданных ед.", command=lambda:build_pie(0))

## Сохранить отчет ##
save_menu = tk.Menu(menubar, tearoff=0)
menubar.add_cascade(label="Сохранить отчет", menu=save_menu)

checkbox=tk.BooleanVar()
radio1=tk.BooleanVar()
radio2=tk.BooleanVar()

switch = False
switch2 = True
switch3 = False
products = None
def option_switch(doSwitch):
    global switch,switch2,switch3, products
    if doSwitch == 1:
        switch = not switch
    elif doSwitch == 2:
        switch2 = not switch2
    elif doSwitch == 3:
        switch3 = not switch3

    param = 1
    if switch2:
        param = 2

    if switch:
        products = quick_sort_(list(read_data())[1], param)
    else:
        products = list(read_data())[1]

    if switch3:
        products = invert_arr(products)

save_menu.add_checkbutton(label="Без сортировки", onvalue=0,offvalue=1, variable=radio1, command=lambda:option_switch(1))
save_menu.add_checkbutton(label="Сортировка по возрастанию", onvalue=1,offvalue=0, variable=radio1, command=lambda:option_switch(1))

sort_menu = tk.Menu(menubar, tearoff=0)
save_menu.add_cascade(label="Параметр сортировки...", menu=sort_menu)
sort_menu.add_checkbutton(label="Сортировка по выручке", onvalue=0,offvalue=1, variable=radio2, command=lambda:option_switch(2))
sort_menu.add_checkbutton(label="Сортировка по кол-ву продаж", onvalue=1,offvalue=0, variable=radio2, command=lambda:option_switch(2))

save_menu.add_checkbutton(label="В обратном порядке", onvalue=1,offvalue=0,variable=checkbox, command=lambda:option_switch(3))
save_menu.add_command(label="Сохранить в PDF", command=lambda:build_report(products))

window.mainloop()






### Вывод
В результате проведенной работы было реализовано оконное приложение с панелью меню. Выполнено чтение данных и расчет общей выручки магазина. Реализовано нахождение товара, который был продан наибольшее количество раз и товара, который принес наибольшую выручку. Также заложена возможность составления и сохранения отчета, содержащего информацию об общей выручке магазина, количестве проданных единиц каждого товара и доле каждого товара в общей выручке в PDF документ. Для решения задач были использованы структуры данных, такие как массивы и хеш-таблицы, а также различные алгоритмы обработки данных, например, быстрая сортировка, расположение элементов массива в обратном порядке, поиск минимального и максимальных значений. Учтены и обработаны в соответствии с требованиями возможные ошибки ввода-вывода.